In [3]:
"""
Scrape các table tuyến bus (table tr: field | value) từ bài VietNamMoi
"""
import re
import json
import time
import requests
from bs4 import BeautifulSoup
from html import unescape

URL = "https://vietnammoi.vn/lich-trinh-cac-tuyen-xe-buyt-tphcm-moi-nhat-2025-thoi-gian-gia-ve-lo-trinh-chi-tiet-202552415731119.htm"
HEADERS = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/120 Safari/537.36"}
OUT_JSON = "../data/bus_routes.json"

# mapping key (dùng substring match, case-insensitive)
FIELD_MAP = {
    "mã số tuyến": "route_code",
    "mã số": "route_code",
    "tuyến": "route_name",
    "lộ trình": "route_path",
    "cự ly": "distance_km",
    "thời gian hoạt động": "service_hours",
    "giá vé": "fare",
    "giá": "fare",
    "số chuyến": "trips_per_day",
    "thời gian chuyến": "trip_time",
    "giãn cách": "headway"
}

# ---------------------------------------------------------------------------------------------------------------------
# util
# ---------------------------------------------------------------------------------------------------------------------
def clean_text(s):
    if s is None:
        return ""
    return re.sub(r'\s+', ' ', unescape(str(s)).strip()).strip()

def request_get_with_retry(url, headers, timeout=30, retries=2, backoff=1.0):
    for attempt in range(retries + 1):
        try:
            r = requests.get(url, headers=headers, timeout=timeout)
            r.raise_for_status()
            return r
        except Exception as e:
            if attempt == retries:
                raise
            time.sleep(backoff * (attempt + 1))

# ---------------------------------------------------------------------------------------------------------------------
# parsing helpers
# ---------------------------------------------------------------------------------------------------------------------
def parse_value_cell(cell):
    """
    Trả:
      - list (items) nếu cell chứa <ul>/<li> hoặc nhiều <p> (ví dụ giá vé)
      - string nếu chỉ 1 đoạn text
    """
    if cell is None:
        return ""

    # nếu có danh sách <ul><li>
    ul = cell.find("ul")
    if ul:
        items = []
        for li in ul.find_all("li"):
            items.append(clean_text(li.get_text()))
        if items:
            return items

    # nếu có nhiều <p> trực tiếp (không đếm nested), trả list
    ps = [p for p in cell.find_all("p", recursive=False) if clean_text(p.get_text())]
    if ps and len(ps) > 1:
        return [clean_text(p.get_text()) for p in ps]

    # nếu có nhiều dòng trong text, nhưng không phải <p> nhiều thì tách dòng và trả list nếu nhiều dòng
    text = clean_text(cell.get_text())
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    if len(lines) > 1:
        return lines

    return text

def map_field_name(key_raw):
    key = clean_text(key_raw).lower()
    for pattern, mapped in FIELD_MAP.items():
        if pattern in key:
            return mapped
    # fallback: slugify
    return re.sub(r'[^a-z0-9]+', '_', key)

def join_if_list_except_fare(val, field_name):
    """
    Nếu val là list và field != 'fare' => join thành chuỗi để dễ xử lý bằng regex
    Giữ list cho 'fare' để giữ nhiều mức giá.
    """
    if isinstance(val, list) and field_name != "fare":
        return " ".join(str(x) for x in val)
    return val

# tách directions/stops từ route_path string
def split_directions(route_path_text):
    if not route_path_text:
        return None, None, [], []
    txt = route_path_text

    # match các khối "Chiều ... : <nội dung>"
    matches = re.findall(r'(?i)(Chiều[^:]{1,80}?)\s*:\s*(.+?)(?=(?:Chiều\s[^:]{1,80}?:)|$)', txt, flags=re.S)
    dir_a = dir_b = None
    if matches:
        if len(matches) >= 1:
            dir_a = clean_text(matches[0][1])
        if len(matches) >= 2:
            dir_b = clean_text(matches[1][1])
    else:
        # fallback: nếu có kí hiệu mũi tên → thì coi cả chuỗi làm direction_a
        if "→" in txt:
            dir_a = clean_text(txt)

    def stops_from(s):
        if not s:
            return []
        parts = re.split(r'\s*[–\-—–]\s*', s)
        return [clean_text(p) for p in parts if clean_text(p)]

    stops_a = stops_from(dir_a)
    stops_b = stops_from(dir_b)
    return dir_a, dir_b, stops_a, stops_b

# ---------------------------------------------------------------------------------------------------------------------
# table parsing
# ---------------------------------------------------------------------------------------------------------------------
def parse_table(tbl, page_url):
    obj = {"raw_html": str(tbl), "page_url": page_url}
    for tr in tbl.find_all("tr"):
        tds = tr.find_all(["td", "th"])
        if len(tds) < 2:
            continue
        key_raw = tds[0].get_text()
        val_cell = tds[1]
        mapped = map_field_name(key_raw)
        val = parse_value_cell(val_cell)
        # nếu val là list và field không phải fare thì join để xử lý sau (regex)
        val = join_if_list_except_fare(val, mapped)
        # gán (nếu key trùng, merge thành list)
        if mapped in obj:
            existing = obj[mapped]
            if isinstance(existing, list):
                if isinstance(val, list):
                    existing.extend(val)
                else:
                    existing.append(val)
            else:
                obj[mapped] = [existing, val]
        else:
            obj[mapped] = val

    # chắc chắn route_path là str trước khi tách direction
    if "route_path" in obj:
        if isinstance(obj["route_path"], list):
            obj["route_path"] = " ".join(obj["route_path"])
        dir_a, dir_b, stops_a, stops_b = split_directions(obj.get("route_path", ""))
        if dir_a:
            obj["direction_a"] = dir_a
        if dir_b:
            obj["direction_b"] = dir_b
        if stops_a:
            obj["stops_a"] = stops_a
        if stops_b:
            obj["stops_b"] = stops_b

    # chuẩn hoá distance
    if "distance_km" in obj and isinstance(obj["distance_km"], str):
        obj["distance_km"] = obj["distance_km"].replace(',', '.').strip()

    # trips_per_day về int nếu có
    if "trips_per_day" in obj and isinstance(obj["trips_per_day"], str):
        m = re.search(r'(\d+)', obj["trips_per_day"])
        if m:
            try:
                obj["trips_per_day"] = int(m.group(1))
            except:
                pass

    return obj

# ---------------------------------------------------------------------------------------------------------------------
# main scraping flow
# ---------------------------------------------------------------------------------------------------------------------
def scrape(url):
    r = request_get_with_retry(url, HEADERS, retries=2, backoff=1.0)
    soup = BeautifulSoup(r.text, "lxml")
    article = soup.find("div", {"itemprop": "articleBody"}) or soup.find("article") or soup.find("div", {"class": "article-content"}) or soup.body
    tables = article.find_all("table")
    results = []
    for tbl in tables:
        parsed = parse_table(tbl, url)
        # giữ nếu có ít nhất route_path hoặc route_code/name
        if parsed.get("route_path") or parsed.get("route_code") or parsed.get("route_name"):
            results.append(parsed)
    return results

if __name__ == "__main__":
    print("Bắt đầu fetch:", URL)
    data = scrape(URL)
    print("Số bảng parsed:", len(data))
    # in 3 mẫu đầu để kiểm tra
    for i, d in enumerate(data[:3], start=1):
        print(f"\n--- mẫu {i} ---")
        print("route_code:", d.get("route_code"))
        print("route_path preview:", (d.get("route_path") or "")[:200])
        print("fare (type):", type(d.get("fare")))
    # lưu file
    with open(OUT_JSON, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print("Đã lưu:", OUT_JSON)


Bắt đầu fetch: https://vietnammoi.vn/lich-trinh-cac-tuyen-xe-buyt-tphcm-moi-nhat-2025-thoi-gian-gia-ve-lo-trinh-chi-tiet-202552415731119.htm
Số bảng parsed: 155

--- mẫu 1 ---
route_code: None
route_path preview: Chiều Bến Thành → Chợ Lớn: Công trường Mê Linh – Đường Thi Sách – Công trường Mê Linh – Đường Tôn Đức Thắng – Đường Hàm Nghi – Đường Trần Hưng Đạo – Đường Nguyễn Tri Phương – Đường Trần Phú – Đường Tr
fare (type): <class 'list'>

--- mẫu 2 ---
route_code: 03
route_path preview: Chiều Bến Thành → Thạnh Xuân: Bến xe buýt Sài Gòn – Đường Phạm Ngũ Lão – Đường Yersin – Đường Trần Hưng Đạo – Đường Hàm Nghi – Đường Hồ Tùng Mậu – Đường nhánh S2 – Đường Tôn Đức Thắng – Đường Hai Bà T
fare (type): <class 'list'>

--- mẫu 3 ---
route_code: 04
route_path preview: Chiều Bến Thành → Bến xe An Sương: Bến xe buýt Sài Gòn – Đường Phạm Ngũ Lão – Đường Yersin – Đường Trần Hưng Đạo – Đường Hàm Nghi – Đường Pasteur – Đường Võ Thị Sáu – Đường Nam Kỳ Khởi Nghĩa – Đường N
fare (type): <class 'list'>
